In [3]:
import os
import numpy
import pandas as pd
import mne

In [4]:
pd.options.display.float_format = '{:.9f}'.format

In [5]:
raw_path = '/workspace/Competition/PSG/01_DATA/raw'

In [6]:
raw_files = os.listdir(raw_path)
raw_files = [x for x in raw_files if not x.startswith('.')]
edf_files = [x for x in raw_files if 'edf' in x]
tsv_files = [x for x in raw_files if 'tsv' in x]
ids = [x.split('.')[0] for x in raw_files]

In [7]:
len(tsv_files)

102

In [63]:
tsv_files

['1_4789.tsv',
 '10_22339.tsv',
 '16_24241.tsv',
 '22_23233.tsv',
 '25_10579.tsv',
 '25_18085.tsv',
 '28_15280.tsv',
 '31_4429.tsv',
 '37_14779.tsv',
 '7_12595.tsv',
 '40_14503.tsv',
 '43_175.tsv',
 '46_11305.tsv',
 '55_6112.tsv',
 '61_19477.tsv',
 '64_14908.tsv',
 '64_25396.tsv',
 '67_5998.tsv',
 '70_20743.tsv',
 '82_20809.tsv',
 '100_23083.tsv',
 '103_12325.tsv',
 '115_22774.tsv',
 '121_23845.tsv',
 '127_19126.tsv',
 '130_5164.tsv',
 '136_4501.tsv',
 '91_15028.tsv',
 '91_18331.tsv',
 '91_3787.tsv',
 '139_20275.tsv',
 '145_1537.tsv',
 '148_568.tsv',
 '151_23734.tsv',
 '154_18469.tsv',
 '157_6391.tsv',
 '160_11500.tsv',
 '160_12856.tsv',
 '160_2740.tsv',
 '163_20485.tsv',
 '166_10357.tsv',
 '181_12274.tsv',
 '184_2509.tsv',
 '187_6160.tsv',
 '190_9877.tsv',
 '202_13666.tsv',
 '205_23176.tsv',
 '211_25741.tsv',
 '211_8290.tsv',
 '220_7708.tsv']

In [64]:
testpath = os.path.join(raw_path, tsv_files[0])
data = pd.read_csv(testpath, delimiter='\t')

In [65]:
data.head()

,onset,duration,description
0,0.335937500,0.000000000,Start Recording
1,0.335937500,0.000000000,"Montage:NCH_PSG_STANDARD, Ref"
2,0.335937500,0.000000000,"Montage:NCH_PSG_STANDARD, Ref"
3,0.460937500,0.000000000,Recording Analyzer - ECG
4,1.007812500,0.000000000,Video Recording ON


In [66]:
data.tail()

,onset,duration,description
1130,34530.261718750,30.000000000,Sleep stage ?
1131,34560.261718750,30.000000000,Sleep stage ?
1132,34590.261718750,30.000000000,Sleep stage ?
1133,34620.261718750,30.000000000,Sleep stage ?
1134,34650.261718750,25.738281250,Sleep stage ?


In [67]:
data.shape

(1135, 3)

In [68]:
data['description'].value_counts()

Sleep stage N2                      453
Sleep stage N3                      246
Sleep stage ?                       104
Sleep stage R                        90
Sleep stage N1                       35
EEG arousal                          34
Sleep stage W                        30
Oxygen Desaturation                  27
move                                 25
Bruxism                               7
Body Position: Supine                 5
Body Position: Prone                  4
Body Position: Left                   4
Soft Snoring                          4
Gain/Filter Change                    4
Body Position: Right                  4
Montage:Channel Test Referential      4
Montage:NCH_PSG_STANDARD              4
Impedance at 10 kOhm                  4
cough                                 3
SNORES                                3
soft snores                           3
Montage:NCH_PSG_STANDARD, Ref         2
EYES CLOSED                           2
Started Analyzer - Sleep Events       2


In [69]:
subdf = data[data['description'].str.contains('stage')]

In [70]:
subdf = subdf.reset_index(drop=True)

In [71]:
subdf

,onset,duration,description
0,5940.261718750,30.000000000,Sleep stage W
1,5970.261718750,30.000000000,Sleep stage W
2,6000.261718750,30.000000000,Sleep stage W
3,6030.261718750,30.000000000,Sleep stage W
4,6060.261718750,30.000000000,Sleep stage W
...,...,...,...
953,34530.261718750,30.000000000,Sleep stage ?
954,34560.261718750,30.000000000,Sleep stage ?
955,34590.261718750,30.000000000,Sleep stage ?
956,34620.261718750,30.000000000,Sleep stage ?


In [72]:
stages = subdf['description']

In [73]:
switch = [0]

i = 0
while i < len(stages)-1:
    cur = stages[i]
    nxt = stages[i+1]
    if cur!=nxt:
        switch.append(i+1)
    i+=1

In [74]:
stagedf = subdf.iloc[switch].reset_index(drop=True)

In [75]:
stagedf = stagedf[['onset','description']]

In [76]:
stagedf.head()

,onset,description
0,5940.261718750,Sleep stage W
1,6300.261718750,Sleep stage N1
2,6390.261718750,Sleep stage N2
3,6810.261718750,Sleep stage N3
4,9330.261718750,Sleep stage N2


In [77]:
stagedf.tail()

,onset,description
65,30960.261718750,Sleep stage N1
66,31050.261718750,Sleep stage ?
67,31080.261718750,Sleep stage N2
68,31560.261718750,Sleep stage W
69,31590.261718750,Sleep stage ?


In [78]:
onset = list(stagedf.onset)

In [79]:
onset

[5940.26171875,
 6300.26171875,
 6390.26171875,
 6810.26171875,
 9330.26171875,
 9360.26171875,
 9750.26171875,
 10710.26171875,
 11970.26171875,
 12690.26171875,
 15150.26171875,
 15360.26171875,
 15990.26171875,
 16020.26171875,
 16770.26171875,
 16830.26171875,
 16950.26171875,
 16980.26171875,
 18450.26171875,
 18480.26171875,
 18840.26171875,
 19470.26171875,
 19590.26171875,
 19620.26171875,
 19650.26171875,
 19680.26171875,
 19710.26171875,
 19740.26171875,
 19800.26171875,
 19890.26171875,
 19920.26171875,
 19980.26171875,
 20850.26171875,
 20880.26171875,
 20970.26171875,
 21180.26171875,
 21240.26171875,
 21300.26171875,
 21360.26171875,
 22830.26171875,
 22860.26171875,
 22980.26171875,
 23010.26171875,
 23040.26171875,
 23070.26171875,
 23370.26171875,
 23790.26171875,
 23850.26171875,
 23880.26171875,
 24480.26171875,
 24510.26171875,
 25470.26171875,
 25530.26171875,
 25920.26171875,
 26010.26171875,
 26040.26171875,
 27060.26171875,
 28080.26171875,
 28110.26171875,
 281

In [80]:
onset[1]*256

1612867.0

In [45]:
import numpy as np

In [46]:
eegpath = '/workspace/Competition/PSG/01_DATA/eeg_7_channel/1_4789.npy'
eegarray = np.load(eegpath)

In [47]:
eegarray.shape

(7, 8877056)

In [48]:
8877056/256

34676.0

In [50]:
30960.261719/256

120.93852233984374

In [51]:
data.head()

,onset,duration,description
0,0.335938,0.0,Start Recording
1,0.335938,0.0,"Montage:NCH_PSG_STANDARD, Ref"
2,0.335938,0.0,"Montage:NCH_PSG_STANDARD, Ref"
3,0.460938,0.0,Recording Analyzer - ECG
4,1.007812,0.0,Video Recording ON


In [52]:
1/256

0.00390625

In [53]:
0.335938/(1/256)

86.000128

In [54]:
34650.26171875 + 25.73828125

34676.0

In [56]:
stagedf.head()

,onset,description
0,5940.261719,Sleep stage W
1,6300.261719,Sleep stage N1
2,6390.261719,Sleep stage N2
3,6810.261719,Sleep stage N3
4,9330.261719,Sleep stage N2


In [57]:
5940.261719/256

23.20414733984375

In [59]:
5940.26171875*256

1520707.0

In [81]:
tsv_files

['1_4789.tsv',
 '10_22339.tsv',
 '16_24241.tsv',
 '22_23233.tsv',
 '25_10579.tsv',
 '25_18085.tsv',
 '28_15280.tsv',
 '31_4429.tsv',
 '37_14779.tsv',
 '7_12595.tsv',
 '40_14503.tsv',
 '43_175.tsv',
 '46_11305.tsv',
 '55_6112.tsv',
 '61_19477.tsv',
 '64_14908.tsv',
 '64_25396.tsv',
 '67_5998.tsv',
 '70_20743.tsv',
 '82_20809.tsv',
 '100_23083.tsv',
 '103_12325.tsv',
 '115_22774.tsv',
 '121_23845.tsv',
 '127_19126.tsv',
 '130_5164.tsv',
 '136_4501.tsv',
 '91_15028.tsv',
 '91_18331.tsv',
 '91_3787.tsv',
 '139_20275.tsv',
 '145_1537.tsv',
 '148_568.tsv',
 '151_23734.tsv',
 '154_18469.tsv',
 '157_6391.tsv',
 '160_11500.tsv',
 '160_12856.tsv',
 '160_2740.tsv',
 '163_20485.tsv',
 '166_10357.tsv',
 '181_12274.tsv',
 '184_2509.tsv',
 '187_6160.tsv',
 '190_9877.tsv',
 '202_13666.tsv',
 '205_23176.tsv',
 '211_25741.tsv',
 '211_8290.tsv',
 '220_7708.tsv']

In [82]:
outpath = '/workspace/Competition/PSG/01_DATA/eeg_7_channel/stage_csv/'

for tsv in tsv_files:
    datapath = os.path.join(raw_path, tsv)
    data = pd.read_csv(datapath, delimiter='\t')
    subdf = data[data['description'].str.contains('stage')]
    subdf = subdf.reset_index(drop=True)
    stages = subdf['description']
    # find state switches
    switch = [0]
    i = 0
    while i < len(stages)-1:
        cur = stages[i]
        nxt = stages[i+1]
        if cur!=nxt:
            switch.append(i+1)
        i+=1
    #
    stagedf = subdf.iloc[switch].reset_index(drop=True)
    stagedf = stagedf[['onset','description']]
    #
    savename = tsv.split('.')[0]
    savepath = os.path.join(outpath, savename+'.csv')
    stagedf.to_csv(savepath)
    print(tsv)

1_4789.tsv
10_22339.tsv
16_24241.tsv
22_23233.tsv
25_10579.tsv
25_18085.tsv
28_15280.tsv
31_4429.tsv
37_14779.tsv
7_12595.tsv
40_14503.tsv
43_175.tsv
46_11305.tsv
55_6112.tsv
61_19477.tsv
64_14908.tsv
64_25396.tsv
67_5998.tsv
70_20743.tsv
82_20809.tsv
100_23083.tsv
103_12325.tsv
115_22774.tsv
121_23845.tsv
127_19126.tsv
130_5164.tsv
136_4501.tsv
91_15028.tsv
91_18331.tsv
91_3787.tsv
139_20275.tsv
145_1537.tsv
148_568.tsv
151_23734.tsv
154_18469.tsv
157_6391.tsv
160_11500.tsv
160_12856.tsv
160_2740.tsv
163_20485.tsv
166_10357.tsv
181_12274.tsv
184_2509.tsv
187_6160.tsv
190_9877.tsv
202_13666.tsv
205_23176.tsv
211_25741.tsv
211_8290.tsv
220_7708.tsv
